In [1]:
#-- The current version is V4.0
#-- THis is upgraded from v3.1 to V4.0
#-- The upgraded version is improved to accomodate reschedules
#--Upgraded by Yusupha Ceesay on 04/07/2024
#-------------------------------------------------------------------------#

In [2]:
from redcap import Project
import pandas as pd
import datetime

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = '63A451E2DA73C4D6A4C64A17B36B9565'
project = Project(api_url, api_key)

#REDCap API configuration for Sensitization
sen_api_url = 'https://redcap.mrc.gm:8443/redcap/api/'
sen_api_key = '008DBC63CA736CB68A3949DEFE43CC5D'
sen_project = Project(sen_api_url,sen_api_key)


### sensitization

In [3]:
In_sen_data=sen_project.export_records(forms=['indigo_sensitisation'])
sen_log_data=pd.DataFrame(In_sen_data)

sen_log_data=sen_log_data[['in_kiang_number','in_name','in_age','in_village', 'in_compound','in_eden_number','in_contact1','in_contact2','in_contact3','in_contact4','in_outcome','in_study_number']]
sen_log_data=pd.DataFrame(In_sen_data)

sen_log_data = sen_log_data[
    (sen_log_data['in_outcome'] == '1') &
    (sen_log_data['in_kiang_number'].notnull()) &
    (sen_log_data['in_study_number'].notnull())
]



In [4]:
#mapping village code to village name
sen_log_data=sen_log_data.rename(columns={'in_study_number': 'con_participantid_q1'})
village_mapping = {
'001':'Dumbuto',
'002':'Sankandi',
'003':'Nioro Jattaba',
'004':'Jattaba',
'005':'Jiffarong',
'006':'Bajana',
'007':'Kuli Kunda',
'008':'Jamaru',
'009':'Brikamanding',
'010':'Kantong Kunda',
'011':'Jali',
'013':'Manduar',
'014':'Bang Kuling',
'015':'Gissay',
'016':'Tankular',
'017':'Joli',
'018':'Kuyang',
'019':'Bantasu',
'020':'Santamba',
'021':'Missira',
'022':'Taborangkoto',
'023':'Burong',
'024':'Jula Kunda',
'025':'Karantaba',
'026':'Mandina',
'027':'Janneh Kunda',
'028':'Kemoto',
'029':'Keneba',
'030':'Batelling',
'031':'Sandeng',
'032':'Wudeba',
'034':'Kenokoto',
'035':'Manari',
'036':'Nineteen',
'040':'WUROKANG',
'041':'KWINELLA SANSANKONO',
'042':'KWINELLA NIA KUNDA',
'043':'TENDABA',
'044':'BUMARR',
'045':'BAMBAKO',
'046':'KUNDONG MARIAYA',
'047':'NEMA',
'048':'KUNDANG NUMU KUNDA',
'049':'KUNDANG FULA KUNDA',
'050':'NEMA KUTA',
'051':'JIRROFF',
'052':'MADINA ANGALLEH',
'053':'JATTA KUNDA',
'054':'MANDINA CENTRAL',
'055':'SARE SARJO',
'056':'SIBETO',
'057':'SARE NDALLA',
'058':'TABANANI',
'060':'WILLINGARA',
'061':'SARE MAMUDU'
}

# Assuming sen_log_data is your DataFrame
sen_log_data['in_village'] = sen_log_data['in_village'].map(village_mapping)

## WITHDRAWALS

In [5]:
#extract study termination data
termination_data = project.export_records(forms=['study_termination'])

#convert data to Dataframe.
term_data=pd.DataFrame(termination_data)

In [6]:
# Specify the fields you want to extract base on logics
df_withdrw=term_data[['con_participantid_q1','st_date','st_reason','redcap_event_name']]
df_withdrw=pd.DataFrame(df_withdrw)

#st_date as a date data type
df_withdrw['st_date'] = pd.to_datetime(df_withdrw['st_date'], errors='coerce')


df_withdrw = df_withdrw[
    (df_withdrw['redcap_event_name'] == 'end_of_study_arm_1') &
    (df_withdrw['st_date'].notnull())
]


In [7]:
Withdrawal_df = pd.DataFrame({
    'con_participantid_q1': df_withdrw['con_participantid_q1'],
    'withdrawal date': pd.to_datetime(df_withdrw['st_date']),
    'Reason': df_withdrw['st_reason'],
    'Status': 'Terminated'
})

# Save the DataFrame to a CSV file
Withdrawal_df.to_csv('Withdrawals.csv', index=False)


USS SCAN DATA

In [8]:
##USS SCAN DATA SET
#extract week 20 completed data
USS = project.export_records(forms=['ultrasound_scan'])

#convert data to Dataframe.
uss=pd.DataFrame(USS)

# Specify the fields you want to extract base on logics for week 20
df_uss=uss[['con_participantid_q1','ultsc_date','ultsc_type_q5','ultsc_reschedule','redcap_event_name']]
df_uss=pd.DataFrame(df_uss)


In [9]:
##EDD LIST
con_uss = project.export_records(forms=['consent','ultrasound_scan'])
con_uss=pd.DataFrame(con_uss)

# Specify the fields you want to extract
df_data_uss=con_uss[['con_participantid_q1', 'con_participant_eden_num_q11', 'ultsc_date', 'ga_weeks','ga_days','con_cluster_q12','redcap_event_name']]

df_data_uss = df_data_uss[
    (df_data_uss['ultsc_date'].notnull())
]
######################################################################################################################################
## EXCLUDE ALL WITHDRAWALS FROM THE DATA
#Read from the ones that complete withdrawals: "Withdrawals.csv"
df_withdrawals = pd.read_csv('Withdrawals.csv')

# Merge the dataframes on 'participant_id'
con_merged_df = pd.merge(df_data_uss, df_withdrawals, on= 'con_participantid_q1', how='inner')

# Filter out the common participant IDs
df_data_uss= df_data_uss[~df_data_uss['con_participantid_q1'].isin(con_merged_df['con_participantid_q1'])]
######################################################################################################################################

# Convert 'ga_weeks' and 'ga_days' columns to numeric type
df_data_uss['ga_weeks'] = pd.to_numeric(df_data_uss['ga_weeks'], errors='coerce')
df_data_uss['ga_days'] = pd.to_numeric(df_data_uss['ga_days'], errors='coerce')

# Calculate today's date
today = pd.Timestamp.today()

# Extract the required columns from the DataFrame
participant_id = df_data_uss['con_participantid_q1']
uss_date = pd.to_datetime(df_data_uss['ultsc_date'])
Ga_weeks = df_data_uss['ga_weeks']
Ga_days = df_data_uss['ga_days']
eden_num = df_data_uss['con_participant_eden_num_q11']

# Calculate the total gestational age in days
total_ga_days = (Ga_weeks * 7) + Ga_days

# Calculate the estimated due date (EDD)
edd = uss_date + pd.to_timedelta(280 - total_ga_days, unit='days')

# Calculate the current gestational age
days_diff = (today - uss_date).dt.days
current_ga = (days_diff + total_ga_days) / 7

event = df_data_uss['redcap_event_name']
cluster = df_data_uss['con_cluster_q12']

## This data frame is mainted to put together the data and maintain the variable name 'con_participantid_q1' for easy exclusion of participants who completed the wk20.


edd_df = pd.DataFrame({
    'con_participantid_q1': participant_id,
    'USS Date': uss_date,
    'EDEN | Infant Number': eden_num,
    'Gestational Age (Weeks)': Ga_weeks,
    'Gestational Age (Days)': Ga_days,
    'Cluster': cluster,
    'Current Gestational Age': current_ga,
    '(Expected)Delivery Date ((E)DD)': edd,

})


In [ ]:
# Get today's date and time
now = datetime.datetime.now()

# Format the current date and time
current_datetime = now.strftime('%Y%m%d_%H%M%S')

# Create the filename with the current date and time
filename = f'EDD List_v2.0_{current_datetime}.csv'

 #Save the filtered DataFrame to a CSV file
edd_df.to_csv(filename, index=False)